In [3]:
import requests
import json
import pandas as pd
from datetime import datetime
import time # Importar la librería time

# URL del endpoint de tu API. Asegúrate de que el contenedor de Docker esté corriendo.
API_URL = "http://localhost:8000/classify-email"
DATASET_PATH = "./Data/spa_dataset.csv"

# --- 1. Leer el dataset ---
try:
    df = pd.read_csv(DATASET_PATH)
    print(f"Dataset '{DATASET_PATH}' cargado exitosamente. Se encontraron {len(df)} correos para probar.\n")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{DATASET_PATH}'. Asegúrate de que está en el mismo directorio.")
    df = pd.DataFrame()

# --- 2. Iterar y testear cada correo del dataset ---
if not df.empty:
    for index, row in df.iterrows():
        print(f"--- Probando correo con ID: {row['id']} ---")

        # Construir el payload con los datos del dataset
        try:
            # Asegurarse de que el formato de fecha sea compatible con ISO 8601
            payload = {
                "id": int(row['id']),
                "subject": row['subject'],
                "email_text": row['email_text']
            }
            
            # Enviar la solicitud POST a la API
            response = requests.post(API_URL, json=payload)
            
            # Imprimir el resultado de la prueba
            print(f"Código de estado: {response.status_code}")
            print("Respuesta:")
            print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        
        except Exception as e:
            print(f"Error al procesar el correo con ID {row['id']}: {e}")

        print("\n" + "="*50 + "\n")
        time.sleep(10) # Pausa de 1 segundo para evitar exceder el límite de la API


Dataset './Data/spa_dataset.csv' cargado exitosamente. Se encontraron 50 correos para probar.

--- Probando correo con ID: 1 ---
Código de estado: 200
Respuesta:
{
  "id": 1,
  "subject": "Reserva tratamiento corporal",
  "email_text": "¿Tienen disponibilidad para un masaje relajante el viernes por la tarde?",
  "clasificacion": "Solicitud de cita",
  "importancia": "media"
}


--- Probando correo con ID: 2 ---
Código de estado: 200
Respuesta:
{
  "id": 2,
  "subject": "Consulta de precios de paquete de faciales",
  "email_text": "¿Cuál es el precio de un masaje de tejido profundo de 60 minutos?",
  "clasificacion": "Consulta de precios/promociones",
  "importancia": "baja"
}


--- Probando correo con ID: 3 ---
Código de estado: 200
Respuesta:
{
  "id": 3,
  "subject": "Solicitud de cita para masaje",
  "email_text": "Me gustaría programar una sesión de reflexología el sábado en la mañana.",
  "clasificacion": "Solicitud de cita",
  "importancia": "media"
}


--- Probando correo con ID

In [13]:
# --- 2. Selecciona el índice del correo que quieres testear ---
# Puedes cambiar este valor para probar diferentes correos del dataset
correo_a_testear_index = 0 # El primer correo del dataset

if not df.empty and correo_a_testear_index < len(df):
    row = df.iloc[correo_a_testear_index]
    print(f"--- Testeando correo con ID: {row['id']} ---")

    # Construir el payload con los datos del dataset
    try:
        payload = {
            "id": int(row['id']),
            "subject": row['subject'],
            "email_text": row['email_text']
        }
        
        # Enviar la solicitud POST a la API
        response = requests.post(API_URL, json=payload)
        
        # Imprimir el resultado de la prueba
        print(f"Código de estado: {response.status_code}")
        print("Respuesta:")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
    
    except Exception as e:
        print(f"Error al procesar el correo con ID {row['id']}: {e}")

else:
    print("El dataset está vacío o el índice seleccionado no es válido.")

--- Testeando correo con ID: 1 ---
Código de estado: 200
Respuesta:
{
  "id": 1,
  "subject": "Reserva tratamiento corporal",
  "email_text": "¿Tienen disponibilidad para un masaje relajante el viernes por la tarde?",
  "clasificacion": "Solicitud de cita",
  "importancia": "media"
}


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

remitente = os.getenv("GMAIL_USER")
contraseña = os.getenv("GMAIL_PASS")

print(f"Usuario de Gmail cargado: {remitente}")
print(f"Contraseña de Gmail cargada: {contraseña}")

Usuario de Gmail cargado: asistente.spa.bella.luna@gmail.com
Contraseña de Gmail cargada: oqnq htda nnwv smex


In [2]:
import smtplib
from email.mime.text import MIMEText

# --- 1. Configuración de credenciales y destinatario ---
# Reemplaza estos valores con tu información.
# ¡IMPORTANTE! Si usas autenticación de dos factores en Gmail,
# debes usar una "clave de aplicación" como contraseña, no la contraseña
# normal de tu cuenta.

destinatario = "seba1carriilo@gmail.com"

# --- 2. Preparar el mensaje ---
asunto = "Prueba de envío de correo desde Python"
cuerpo = """
Este es un mensaje de prueba para verificar que la funcionalidad de envío de
correos con smtplib está funcionando correctamente.

Si recibes este correo, significa que la configuración de las credenciales y
el script son correctos.
"""

mensaje = MIMEText(cuerpo)
mensaje["Subject"] = asunto
mensaje["From"] = remitente
mensaje["To"] = destinatario

# --- 3. Conectarse al servidor SMTP de Gmail y enviar el correo ---
servidor_smtp = "smtp.gmail.com"
puerto_smtp = 587

print("Intentando enviar el correo...")
try:
    with smtplib.SMTP(servidor_smtp, puerto_smtp) as server:
        server.starttls()  # Habilita el cifrado TLS
        server.login(remitente, contraseña)
        server.sendmail(remitente, destinatario, mensaje.as_string())
        print("¡Correo enviado exitosamente!")
except smtplib.SMTPAuthenticationError:
    print("Error de autenticación: Verifica que tu usuario y contraseña (o clave de aplicación) sean correctos.")
except Exception as e:
    print(f"Ocurrió un error inesperado al enviar el correo: {e}")


Intentando enviar el correo...
¡Correo enviado exitosamente!
